<a href="https://colab.research.google.com/github/brenoslivio/SME0828_DataScience/blob/master/Projects/3%20-%20Classifica%C3%A7%C3%A3o/Projeto3_Classificacao_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SME0828 - Introdução à Ciência de Dados
# Projeto 3: Classificação 

### Alunos

Aline Fernanda da Conceição, Nº USP: 9437275

Breno Lívio Silva de Almeida, Nº USP: 10276675

Matheus Victal Cerqueira, Nº USP: 10276661

***Universidade de São Paulo, São Carlos, Brasil.***

## Exercícios sugeridos para a atividade avaliativa 3

Os seguintes exercícios foram sugeridos para entrega em 03 de novembro de 2020 para a disciplina SME0828 - Ciência de Dados oferecida pelo Instituto de Ciências Matemáticas e de Computação (ICMC) da Universidade de São Paulo (USP), a qual foi ministrada pelo professor Francisco Aparecido Rodrigues. Vamos trabalhar com o conjunto de dados [*Star dataset to predict star types*](https://www.kaggle.com/deepu1109/star-dataset). O dataset tem várias informações sobre características de estrelas como temperatura absoluta, luminosidade relativa, tipos de estrelas, entre outros. Vamos classificar os dados considerando vários classificadores conhecidos.



---



## Sumário

1. Desenvolvimento

  1.1 [Exercício 1](#ex1)

  1.2 [Exercício 2](#ex2)

  1.3 [Exercício 3](#ex3)

  1.4 [Exercício 4](#ex4)

  1.5 [Exercício 5](#ex5)

  1.6 [Exercício 6](#ex6)



---



## Desenvolvimento



### <a name="ex1"></a> 1 - Considere a base de dados sobre doenças cardíacas:https://www.kaggle.com/ronitf/heart-disease-uci 
### Faça o pré-processamento dos dados e classifique os pacientes de acordo com a variável “target”.  Considere os classificadores: Bayesiano paramétrico, Bayesiano não-paramétrico e Naive Bayes.

Foi considerada a base de dados de estrelas comentadas antes. Vamos trabalhar com a classificação para a variável Star color.

In [273]:
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import pandas as pd
import statistics

dataStar = pd.read_csv('https://raw.githubusercontent.com/brenoslivio/SME0828_DataScience/master/Projects/data/starsDataset.csv', 
        dtype={
          "Temperature (K)": np.int64,
          "Luminosity(L/Lo)": np.float64,
          "Radius(R/Ro)": np.float64,
          "Absolute magnitude(Mv)": np.float64,
          "Star type": str,
          "Star color": str,
          "Spectral Class": str
        },
        na_values="",)

dataStar.replace("", np.nan, inplace=True)

dataStar.dropna(inplace=True)

# armazena os nomes das classes
classes = np.array(pd.unique(dataStar[dataStar.columns[-3]]), dtype=str)  

attributes = list(dataStar.columns)

dataStar.sample(5)

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Star color,Spectral Class
238,9235,404940.00000,1112.0000,-11.23,5,White,A
204,18340,0.00134,0.0124,11.22,2,Blue,B
39,5587,0.81900,0.9900,5.03,3,yellow-white,F
36,6757,1.43000,1.1200,2.41,3,yellow-white,F
68,2774,0.00036,0.1180,17.39,0,Red,M


As classes para a cor das estrelas

In [274]:
classes

array(['0', '1', '2', '3', '4', '5'], dtype='<U1')

In [275]:
data = dataStar.to_numpy()
data

array([[3068, 0.0024, 0.17, ..., '0', 'Red', 'M'],
       [3042, 0.0005, 0.1542, ..., '0', 'Red', 'M'],
       [2600, 0.0003, 0.102, ..., '0', 'Red', 'M'],
       ...,
       [8829, 537493.0, 1423.0, ..., '5', 'White', 'A'],
       [9235, 404940.0, 1112.0, ..., '5', 'White', 'A'],
       [37882, 294903.0, 1783.0, ..., '5', 'Blue', 'O']], dtype=object)

In [276]:
nrow,ncol = data.shape
y = data[:,-3]
X = data[:,0:ncol-3]

Normalizando os dados

In [277]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

print('Dados transformados:')
print('Media: ', np.mean(X, axis = 0))
print('Desvio Padrao:', np.std(X, axis = 0))

Dados transformados:
Media:  [-1.48029737e-17  2.96059473e-17 -1.57281595e-17 -3.51570624e-17]
Desvio Padrao: [1. 1. 1. 1.]


In [265]:
classes

array(['0', '1', '2', '3', '4', '5'], dtype='<U1')

In [278]:
from sklearn.model_selection import train_test_split
p = 0.8 # fracao de elementos no conjunto de treinamento
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = p, random_state = 42)

In [267]:
np.where(y_train == classes[0])

(array([  1,   3,  13,  14,  15,  17,  25,  28,  33,  59,  60,  74,  84,
         85,  89,  91,  93,  95,  96, 100, 109, 113, 118, 124, 132, 144,
        151, 163, 165, 169, 182, 185]),)

In [279]:
from scipy.stats import multivariate_normal

#matrix to store the probabilities
P = pd.DataFrame(data = np.zeros((x_test.shape[0], len(classes))), columns = classes) 

Pc = np.zeros(len(classes)) #fraction of elements in each class
for i in np.arange(0, len(classes)):
    elements = tuple(np.where(y_train == classes[i]))
    Pc[i] = len(elements)/len(y_train)
    Z = x_train[elements,:][0]
    m = np.mean(Z, axis = 0)
    cv = np.cov(np.transpose(Z))
    for j in np.arange(0,x_test.shape[0]):
        x = x_test[j,:]
        pj = multivariate_normal.pdf(x, mean=m, cov=cv, allow_singular=True)
        P[classes[i]][j] = pj*Pc[i]


In [280]:
print(P)

                0              1  ...              4             5
0    0.000000e+00   0.000000e+00  ...  6.159614e-244  2.730161e-54
1    5.855497e+02   2.365505e-02  ...   0.000000e+00  1.073849e-81
2   2.590247e-296   4.033989e-16  ...   2.558474e-95  1.213054e-28
3    0.000000e+00   0.000000e+00  ...   5.914496e-03  1.130813e-10
4    0.000000e+00   0.000000e+00  ...   6.954607e-03  1.471600e-09
5    0.000000e+00   0.000000e+00  ...   0.000000e+00  7.127491e-05
6    0.000000e+00   0.000000e+00  ...   0.000000e+00  6.537956e-06
7   4.335083e-150  6.243537e-318  ...  2.238164e-252  1.785297e-55
8    2.533469e+02   5.475704e-01  ...   0.000000e+00  2.667561e-75
9   2.521004e-246   0.000000e+00  ...  1.331486e-245  1.825179e-54
10   0.000000e+00   0.000000e+00  ...   1.006093e-02  3.988185e-09
11   0.000000e+00   0.000000e+00  ...  8.796195e-207  1.135201e-48
12   0.000000e+00   0.000000e+00  ...   0.000000e+00  6.252431e-04
13   0.000000e+00   0.000000e+00  ...   2.680044e-02  1.319782

In [281]:
y_pred = []
#np.array(test_x.shape[0], dtype=str)
for i in np.arange(0, x_test.shape[0]):
    c = np.argmax(np.array(P.iloc[[i]]))
    y_pred.append(classes[c])
y_pred = np.array(y_pred, dtype=str)
print(y_pred)

['2' '0' '3' '4' '4' '5' '5' '2' '0' '2' '4' '2' '5' '4' '5' '5' '0' '1'
 '3' '0' '0' '1' '5' '3' '5' '3' '1' '2' '5' '3' '3' '4' '2' '0' '1' '0'
 '1' '5' '1' '5' '4' '5' '4' '0' '3' '0' '4' '3']


In [282]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_pred, y_test)
print('Accuracy:', score)

Accuracy: 0.9791666666666666


### <a name="ex2"></a> 2 - No classificar não-paramétrico, verifique o efeito do hiperparâmetro h na classificação dos dados de diabetes, encontrando seu melhor valor: https://www.kaggle.com/uciml/pima-indians-diabetes-database
